notebook-qNN.ipynb

Notebook that define the necessary functions to run a qNN and runs it.

Dependencies:
- Uses qiskit.circuit.library module to generate a circuit with all the possible inputs
- Uses qiskit.primitives module to run the circuit

Since:
- 02/2025

Authors:
- Pedro C. Delbem. <pedrodelbem@usp.br>

#imports

In [7]:
from single_qubit_qNN import qNN as sqNN
from two_qubit_qNN import qNN as tqNN

#single qubit neuron

##generate qNN circuit

In [9]:
single_qubit_qNN = generate_qNN_circuit([0,0],[0,0,0])
single_qubit_qNN.get_current_circuit().draw()

NameError: name 'generate_qNN_circuit' is not defined